## Clustering d’article de journaux 

Scrapper plusieurs articles de différents journaux 
Construisez un modèle vous permettant de regrouper ces articles x journaux par secteur : Tech/Economie etc…

### Introduction

### 1ère étape: scrapping
Nous allons dans un premier temps scrapper des articles de journaux, il faut essayer de choisir des articles diverse et varier.

In [2]:
from bs4 import BeautifulSoup
import urllib3
import urllib
import requests
import csv
import pandas as pd

On prend un site d'actualités au hazard comme le monde.fr par exemple. Puis un cherche à faire ressortir tous les liens des articles. Pour cela il faut éliminer les liens qui ne nous intéresse pas, c'est à dire les liens qui ne redirigent pas vers un article.

In [3]:
url = "https://www.lemonde.fr/" # site internet du mande 

def getLinks(url):
    links=[]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a'): # boucle qui récupère tous les liens de la page
        # en regardant les liens des articles, on remarque que tous les liens se terminent par .html
        # et ont comme préfixe : "https://www.lemonde.fr"
        if 'www.lemonde.fr' in link.get('href') and '.html' in link.get('href') : 
            links.append(link.get('href'))
    return(links)

getLinks("https://www.lemonde.fr/")

#La fonction renvoie la liste des articles
#Les conditions sont hardcodées et définies par rapport au site du monde
#Cette fonction ne fonctionne pas néscéssairement sur d'autres sites

['https://www.lemonde.fr/economie/article/2021/04/06/quatre-nouveaux-milliardaires-francais-dont-le-pdg-de-moderna-entrent-au-classement-forbes_6075750_3234.html',
 'https://www.lemonde.fr/economie/article/2021/04/01/telecommunications-huawei-de-plus-en-plus-chinois-de-moins-en-moins-global_6075228_3234.html',
 'https://www.lemonde.fr/economie/article/2021/04/06/smartphones-la-chine-fait-subir-aux-sud-coreens-le-meme-sort-que-ces-derniers-ont-inflige-aux-japonais-a-partir-des-annees-2000_6075700_3234.html',
 'https://www.lemonde.fr/idees/article/2021/04/06/mayotte-un-simple-copier-coller-de-ce-qui-se-fait-ailleurs-en-france-ne-peut-constituer-une-strategie-de-developpement-pour-ce-territoire_6075754_3232.html',
 'https://www.lemonde.fr/international/video/2021/03/07/birmanie-pourquoi-l-armee-s-est-elle-toujours-sentie-autant-menacee-par-aung-san-suu-kyi_6072261_3210.html',
 'https://www.lemonde.fr/international/video/2021/04/04/mercenaires-russes-wagner-enquete-video-sur-l-armee-fantom

On cherche maintenant à voyager entre les liens du site pour aller scrapper les articles. Dans un premier temps on récupère les titres de chaque article.

In [4]:
def getTitle(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.title.string

#for link in getLinks("https://www.lemonde.fr/"):
#    print(getTitle(link))


On souhaite maintenant récupèrer en plus du titre, la préface et enfin l'article en lui même.
Le script suivant nous permet de récupèrer la préface de l'article suivant.

In [5]:
requete = ("https://www.lemonde.fr/economie/article/2021/04/01/telecommunications-huawei-de-plus-en-plus-chinois-de-moins-en-moins-global_6075228_3234.html")
html = urllib.request.urlopen(requete).read()
#soup = BeautifulSoup(html)

soup = BeautifulSoup(html, 'html.parser')  # html contains above data
elements = soup.find(class_='article__desc')  # The class we're searching for is "class_c"
elements = elements.get_text()
print(elements)


    Le groupe chinois a enregistré une progression de son chiffre d’affaires de près de 4 % en 2020, grâce au déploiement de la 5G dans l’empire du Milieu. Hors de son pays d’origine, en revanche, il voit son activité freinée par les sanctions politiques. 


Le script suivant nous permet de récuèrer le contenu de l'article en question. 

In [4]:
requete = ("https://www.lemonde.fr/economie/article/2021/04/01/telecommunications-huawei-de-plus-en-plus-chinois-de-moins-en-moins-global_6075228_3234.html")
html = urllib.request.urlopen(requete).read()
#soup = BeautifulSoup(html)

soup = BeautifulSoup(html, 'html.parser')  # html contains above data
elements = soup.find(class_='article__content')  # The class we're searching for is "class_c"
elements = elements.get_text()
print(elements)


      Un magasin Huawei, à Pékin, le 25 mai 2020. NICOLAS ASFOURI / AFP    Il y a précisément un an, alors que Huawei annonçait ses résultats annuels, le géant chinois des télécommunications, visé par des sanctions américaines, avait placé 2020 – avec un peu de dramatisation – sous le signe de la survie. Il n’avait alors probablement pas mesuré l’impact de la pandémie de Covid-19 sur l’économie mondiale. Dès lors, ses performances pour l’exercice 2020 ont de quoi surprendre.  Article réservé à nos abonnés  Lire aussi  2020, l’année où Huawei a mis un genou à terre   Mercredi 31 mars, le conglomérat de Shenzhen a levé le voile sur un chiffre d’affaires en progression de 3,8 % par rapport à 2019, à 891,4 milliards de yuans (115,5 milliards d’euros), et un bénéfice net en hausse de 3,2 %, à 64,6 milliards de yuans. C’est toutefois un net coup de frein par rapport à la croissance de ses revenus en 2019 (+ 19,1 %) Cette résistance face à la crise, Huawei la doit pour beaucoup à son marché i

Nous allons maintenant enregistrer les articles dans un dataset, pour avoir une base de données à notre disposition, base de données avec laquelle nous pourons travailler. La base de données, va s'organiser de la manière suivante pour chaque article: url, titre, préface, contenu.

Les deux fonctions suivantes permettent de récupèrer respectivement la préface et le contenu des articles pour chaque lien récupèré depuis le site internet du monde.

In [6]:
def getDesc(url):
    html = urllib.request.urlopen(url).read()
    #soup = BeautifulSoup(html)
    try :
        soup = BeautifulSoup(html, 'html.parser')  # html contains above data
        elements = soup.find(class_='article__desc')  # The class we're searching for is "class_c"
        elements = elements.get_text()
        return(elements)
    except:
        return("desc unavaible")

#for url in getLinks("https://www.lemonde.fr/"):
#    print(getDesc(url))


In [7]:
def getContent(url):
    html = urllib.request.urlopen(url).read()
    #soup = BeautifulSoup(html)
    try :
        soup = BeautifulSoup(html, 'html.parser')  # html contains above data
        elements = soup.find(class_='article__content')  # The class we're searching for is "class_c"
        elements = elements.get_text()
        return(elements)
    except:
        return("content unavaible")

#for url in getLinks("https://www.lemonde.fr/"):
#   print(getContent(url))

In [8]:

def createDataFrame(urls):
    #Create an empty DataFrame object
    df = pd.DataFrame(columns=['Url','Title','Desc','Content'])
    
    for url in urls:
        df = df.append({'Url' : url ,'Title': getTitle(url) , 'Desc' : getDesc(url), 'Content' : getContent(url)}, ignore_index = True)

    
    return(df)

df = createDataFrame(getLinks("https://www.lemonde.fr/"))
df.head()
#createDataFrame(getLinks("https://www.lemonde.fr/")).to_csv('journaux.csv',sep='\t', encoding='utf-8')

,Url,Title,Desc,Content
0,https://www.lemonde.fr/economie/article/2021/0...,"Quatre nouveaux milliardaires français, dont l...",Détenant 8 % du capital de Moderna Therape...,La crise due à la pandémie de Covid-19 n’a pa...
1,https://www.lemonde.fr/economie/article/2021/0...,"Télécommunications : Huawei, de plus en plus c...",Le groupe chinois a enregistré une progres...,"Un magasin Huawei, à Pékin, le 25 mai 20..."
2,https://www.lemonde.fr/economie/article/2021/0...,Smartphones : « la Chine fait subir aux Sud-Co...,LG a annoncé l’arrêt de son activité téléphoni...,"Devant le siège de LG, à Séoul, le 5 avr..."
3,https://www.lemonde.fr/idees/article/2021/04/0...,Mayotte : « un simple « copier-coller » de ce ...,"Mieux insérer les femmes, les jeunes et les na...",Tribune. Violence et pauvreté vont souvent de...
4,https://www.lemonde.fr/international/video/202...,Du coup d’État à la résistance : comprendre la...,vidéo Des manifestants prodémocratie lutte...,"Le 1er février 2021, les promesses de transit..."


La fonction suivante a pour but de récupèrer les labels, c'est à dire les mots clés avec lesquels sont décrits les articles sur le site du monde lui même. Je souhaites remettre les labels par la suite dans le dataframe afin de les comparer, mais pour le moment la fonction n'est pas parfaitement au point car elle renvoie uniquement le premier label même lorsque qu'il y en a plusieurs.

In [7]:
requete = ("https://www.lemonde.fr/idees/article/2021/04/03/la-pratique-de-la-peche-au-vif-n-est-plus-conforme-avec-le-droit_6075463_3232.html")
html = urllib.request.urlopen(requete).read()
#soup = BeautifulSoup(html)

soup = BeautifulSoup(html, 'html.parser')  # html contains above data
elements = soup.find_all(class_='breadcrumb')  # The class we're searching for is "class_c"


#li = soup.find('li', {'class': 'breadcrumb'})
#children = li.findChildren("a" , recursive=False)
#for child in children:
    #print(child)
    

ul = soup.find(class_ = 'breadcrumb')
children = ul.findChildren()
for child in children:
    print (child)
    
#elements = elements.get_text()
#print(elements)

<li class="breadcrumb__parent js-breadcrumb"> <a class="logo__idees logo__idees--article" href="/idees/">  Opinions </a> </li>
<a class="logo__idees logo__idees--article" href="/idees/">  Opinions </a>


In [33]:
requete = ("https://www.lemonde.fr/idees/article/2021/04/03/la-pratique-de-la-peche-au-vif-n-est-plus-conforme-avec-le-droit_6075463_3232.html")
html = urllib.request.urlopen(requete).read()
#soup = BeautifulSoup(html)

soup = BeautifulSoup(html, 'html.parser')  # html contains above data
#elements = soup.find_all(class_='breadcrumb')  # The class we're searching for is "class_c"

elements =soup.find("li", { "class" : "test" }).findAll("a", recursive=False)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [35]:
url = 'https://www.lemonde.fr/economie/article/2021/04/02/le-gendarme-boursier-tacle-suez-sur-sa-defense-contre-l-opa-de-veolia_6075436_3234.html'

text = getContent(url)
print(text)


      Logos accolés des deux champions de l’eau et des déchets, dont le premier, Veolia, a lancé le 7 février 2021 une OPA hostile contre le deuxième, Suez. KENZO TRIBOUILLARD, ERIC PIERMONT / AFP    Le gendarme de la Bourse a sorti son sifflet. L’Autorité des marchés financiers (AMF) a pris position, vendredi 2 avril, contre le dispositif mis en place par Suez pour contrer l’OPA de Veolia. Le géant de l’environnement assiégé a immédiatement réagi en dénonçant « une prise de position incompréhensible et favorisant une nouvelle fois Veolia ». Dans le camp de l’attaquant, en revanche, la satisfaction était de mise, même si, à ce stade, l’AMF n’a assorti sa communication d’aucune injonction. Quels sont les reproches du gendarme boursier ? Pour comprendre, il faut se rappeler que la défense de Suez repose largement sur l’existence d’une fondation de droit néerlandais, créée, en septembre 2020, afin de rendre Suez Eau France inaliénable. Objectif revendiqué : gêner Veolia qui, en cas de suc

### 2ème étape : L'analyse 

Nous allons utiliser la librairie nltk

In [30]:
import nltk
#nltk.download() 
# il faut lancer la commande commentée puis relancer le noyau pour que la commande nltk.word_tokenize fonctionne

import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.base import get_data_home

from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt

Dans un premier temps, on utilise la librairie sur un texte seul

In [26]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;.]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('french'))
SHORTWORD = re.compile(r'\W*\b\w{1,3}\b')

#stop_words_lst = ['cell','spectral','data','double','bond','barrier','soluble', 'cells', 'performed using']

def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = str(text).lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = re.sub(r'\b[a-zA-Z]\b','',str(text))
    text = re.sub('\d+','',text) 
    text = SHORTWORD.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) 
    #for w in stop_words_lst:
        #pattern = r'\b'+w+r'\b'
        #text = re.sub(pattern, '', str(text))

    return text

In [27]:
url = 'https://www.lemonde.fr/economie/article/2021/04/02/le-gendarme-boursier-tacle-suez-sur-sa-defense-contre-l-opa-de-veolia_6075436_3234.html'

text = getContent(url)

tokens = nltk.word_tokenize(text)
#print (tokens)

def extracting(url):
    text = clean_text(getContent(url))
    text_tokens = word_tokenize(text)

    text_i=[text]

    # Getting bigrams and trigrams
    vectorizer = CountVectorizer(ngram_range =(2, 3))
    X1 = vectorizer.fit_transform(text_i)
    features = (vectorizer.get_feature_names())

    X2 = vectorizer.fit_transform(text_i)
    scores = (X2.toarray())

    sums = X2.sum(axis = 0)
    data1 = []
    for col, term in enumerate(features):
        data1.append( (term, sums[0, col] ))
    ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
    words = (ranking.sort_values('rank', ascending = False))
    return ("\n\nWords : \n", words.head(50))

extracting('https://www.lemonde.fr/economie/article/2021/04/02/le-gendarme-boursier-tacle-suez-sur-sa-defense-contre-l-opa-de-veolia_6075436_3234.html')


('\n\nWords : \n',
                                     term  rank
 172                     droit nerlandais     2
 469                          suez france     2
 95               conseil dadministration     2
 207                      fondation droit     2
 208           fondation droit nerlandais     2
 331                    offres surenchres     2
 72                       cette fondation     2
 361                prcise lautorit reste     1
 360                      prcise lautorit     1
 362                       premier veolia     1
 351               portent atteinte rgles     1
 359             prcisait contours accord     1
 358                    prcisait contours     1
 357       pourraient permettre dissoudre     1
 356                 pourraient permettre     1
 355              position vendredi avril     1
 354                    position vendredi     1
 353  position incomprhensible favorisant     1
 352             position incomprhensible     1
 0                   

In [34]:
import string
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    4. Remove words
    '''
    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc =  [word.lower() for word in nopunc.split() if word not in stopwords.words('english')]
    return [stemmer.lemmatize(word) for word in nopunc]

#Vectorisation : -
tfidfconvert = TfidfVectorizer(analyzer=text_process,ngram_range=(1,3)).fit(X_train)

X_transformed=tfidfconvert.transform(X_train)

# Clustering the training sentences with K-means technique

from sklearn.cluster import KMeans
modelkmeans = KMeans(n_clusters=60, init='k-means++', n_init=100)
modelkmeans.fit(X_transformed)

Sum_of_squared_distances = []
K = range(1,150)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(X_transformed)
    Sum_of_squared_distances.append(km.inertia_)


import matplotlib.pyplot as plt

plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()



NameError: name 'X_train' is not defined

Par manque de temps, je n'ai pas pu finir ce que je souhaitais faire, je vais donc lexpliquer ici.


Points à finir dans la première partie:
* enregistrer le dataset sur un fichier CSV proprement
* Scrapper les labels des articles de journaux

Points à finir dans la seconde partie:
* ce que je souhaiatais faire par la suite, était d'utiliser le code ci-dessus afin de faire apparaitre la courbe d'elbow qui m'aurait permis de déterminer le nombre de cluster optimal.
* généraliser le processus a tous les articles du dataset
* comparer les clusters aux labels et voir si les prédictions sont correctes, faire la même chose avec la préface et voir si les clusters correspondent*
* J'aurais adapté quelque chose de similaire à ce qui est décrit dans l'article suivant : https://towardsdatascience.com/applying-machine-learning-to-classify-an-unsupervised-text-document-e7bb6265f52